<img src="figures/logos.png" style="float: centre;" width="800"/>

# 震间实用代码

## 为 2022 年 COMET InSAR 研讨会撰写。


欢迎来到关于震间应变积累的 COMET 实践。

此实用程序已在最新版本的 Chrome 和 Firefox 中进行了测试，但应该适用于大多数现代浏览器

### 代码交互

要运行包含代码的单元格，你可以按上面栏中的“运行”，或按“Shift+Enter”。

“#”表示代码中的注释。 

##################################### <br />
被多个“#”包围的代码块(比如这个)表示可以尝试改变其值的变量。虽然您可以进一步试验整个代码体(我鼓励您在课程结束后这样做)，但我们帮助您调试可能遇到的任何错误的能力有限。仅更改指示值应该不会导致重大错误。 <br />
#####################################

如果出现“死内核”错误，请在顶部栏的“内核”下重新启动内核，或者重新打开新鲜的活页夹。此处的任何更改都不会影响原始活页夹，因此如果有任何损坏，请随时重新加载它。

### Acknowledgements
Detailed guidance on the content was provided by [John Elliott](https://environment.leeds.ac.uk/see/staff/1248/dr-john-elliott). Advice on the computational requirements of the practical was provided by [Richard Rigby](https://environment.leeds.ac.uk/see/staff/2698/richard-rigby). This notebook was written using resources from the Tectonophysics ([Tim Wright](https://environment.leeds.ac.uk/see/staff/1613/professor-tim-wright)) and Inverse Theory ([Phil Livermore](https://environment.leeds.ac.uk/see/staff/1381/dr-phil-livermore)) undergraduate modules at the University of Leeds. Thanks to everyone for their contributions.

[Andrew Watson](https://environment.leeds.ac.uk/see/pgr/2311/andrew-watson) - 2021

In [ ]:
# 导入所需的模块。在继续之前运行这个程序。.

# 这些用于执行分析
import numpy as np
from scipy import interpolate
from scipy import stats
import statistics
import pyproj
import subprocess as subp
from pathlib import Path

# 这是我们自己的函数库，可以在与本笔记本相同的目录中找到
import interseis_lib as lib

# 这个包是一般绘图
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# 这是额外的科学彩色地图，可以在这个网址看 "https://www.fabiocrameri.ch/colourmaps/"
from cmcrameri import cm

# 这些包只需要用于最后的多变量图
import seaborn as sns
import pandas as pd

# 如果在笔记本运行时进行了更改，则会刷新interseis_lib的导入
import importlib
importlib.reload(lib)

## 0. 介绍

地震周期是地震如何发生在断层上的一个模型。它包括三个阶段:
- **震间** - 断层向地表闭锁，但在外力(即板块运动)的驱动下，在深度上积累应变。
- **同震** - 这种应变是通过断层先前闭锁部分的滑动释放出来的。
- **震后** - 震后机制，如余震或粘弹性松弛。

我们将把重点放在震间期。图1显示了地面在震间期(中图)和同震期(右图)如何变形的示意图。剖面A - A’在中间板块(arctan函数的形状)显示了一个特征性的震间信号，在断层迹线处，地面位移从远场减小到零.

<img src="figures/earthquakeCycle.png" style="float: centre;" width="600"/>

*Figure 1: 在震间期和同震期预期的地面变形示意图。选自 Elliott (2009).*

大地测量学家对这个震间期很感兴趣，因为它提供了有关应变积累速率和断层闭锁部分大小的信息，这是了解断层地震潜力和地壳应变分布的重要参数。

我们可以通过考虑嵌入弹性半空间内的螺位错的稳定滑移来模拟震间变形(Savage和Burford, 1973, Wright等人，2013)。在这个简单的模型中，与断层平行的地表速度$v$是与断层垂直距离$x$、滑移率$s$和锁定深度$d$的函数:

\begin{equation}
\normalsize v(x) = \frac{s}{\pi}tan^{-1}\left(\frac{x}{d}\right) + c \qquad \qquad (1)
\end{equation}

Figure 2 给出了螺旋位错模型的示意图.

<img src="figures/screwDisc.png" style="float: centre;" width="600"/>

*Figure 2: 螺旋位错模型示意图。转载自 Vernant (2015).*

LiCS对土耳其北安那托利亚断层(NAF)的大量InSAR数据进行了处理。我们将使用其中两个LiCSBAS时间序列(087A_04904_121313和167D_04884_131212，如图3所示)来估计断层平行速度，然后我们将使用它们来估计公式1中的参数值。

<img src="figures/frameMap.png" style="float: centre;" width="600"/>

*Figure 3: 我们的两个LiCS frame的位置覆盖在断层(红线)从全球地震模型 (https://github.com/GEMScienceTools/gem-global-active-faults).*

这个代码分为以下几个部分:

**1)** 生成一个简单的合成螺旋位错模型，并研究我们可能期望InSAR测量的内容。

**2)** 加载我们的LiCSBAS速度和其他数据文件。

**3)** 在北速度为零的假设下，将我们的两个视线速度图分解为东速度和垂直速度，在断层垂直速度为零的假设下，分解为断层平行速度和垂直速度。这包括重新采样我们的速度到一个统一的网格和设置一个共同的参考点。

**4)** 将断层平行速度投影到断层垂直剖面上。

**5)** 执行正演模型来探索我们的剖面速度。

**6)** 执行贝叶斯反演来估计公式1中的参数及其相关的不确定性。

**7)** 扩展-在我们的模型中纳入了地表断层蠕变。

**8)** 扩展-估计剪切应变率。

对于这种一般方法的一个例子，请看我们对伊朗近期主要断层的预印本:
https://www.essoar.org/doi/abs/10.1002/essoar.10507719.1

## 1. 合成螺位错模型

我们首先用螺旋位错模型生成一些合成速度。

In [ ]:
# 设置震间断层滑动速率s和锁定深度d
#####################################
s = 20 # mm/yr          
d = 20 # km             
#####################################

# 设置x坐标(断层垂直距离)
xmin, xmax, xint = -200, 200, 2 # km
x = np.arange(xmin, xmax+xint, xint)

# 运行螺旋位错模型。0只是一个静态偏移量，我们现在可以忽略它。
# 我们把所有的单位都转换成SI(米)，这样它们就一致了。
v = lib.screw_disc(x*1000, s/1000, d*1000, 0)

# 我们还将生成一个s = 10毫米/年和d = 10公里的模型作为参考解决方案
v_ref = lib.screw_disc(x*1000, 10/1000, 10*1000, 0)

# 扩展到二维，假设我们的断层在东西方向
v_grid = np.tile(v, (len(x),1)).T


# 绘制结果
# 让我们把第一个绘图部分分解一下

# 为绘图创建底图和坐标
fig, axs = plt.subplots(1,2,figsize=(15,5))

# 将模型速度绘制成一条线到第一个轴/子图
axs[0].plot(x, v*1000, color="blue", label='Your model: \ns = ' + str(s) + ' mm/yr, d = ' + str(d) + ' km') 

# 绘制参考模型
axs[0].plot(x, v_ref*1000, color="red", label='Reference model: \ns = 10 mm/yr, d = 10 km')

# 在x=0处(断层轨迹)和y=0处绘制虚线
axs[0].plot([0, 0],[axs[0].get_ylim()[0], axs[0].get_ylim()[1]], color='grey', linestyle='dashed')
axs[0].plot([axs[0].get_xlim()[0], axs[0].get_xlim()[1]], [0, 0], color='grey', linestyle='dashed')

# 添加图例
axs[0].legend(fontsize=12)

# 为x和y轴设置标签
axs[0].set_xlabel('Fault-perpendicular distance (km)')
axs[0].set_ylabel('Fault-parallel velocity (mm/yr)')


# 二维速度的彩色图，现在在我们的第二个轴/子图上
im = axs[1].imshow(v_grid*1000, extent=[xmin, xmax, xmin, xmax], cmap=cm.vik)

# 添加一个带标签的颜色条
plt.colorbar(im, ax=axs[1], label="Fault-parallel velocity (mm/yr)")

# 将断层添加为实线
axs[1].plot([xmin, xmax], [0, 0], color='black', label='Fault trace') 

# 将剖面添加为虚线
axs[1].plot([0, 0],[xmin, xmax], color='black', linestyle='dashed', label="Profile line")

# 添加图例
axs[1].legend(fontsize=12)

# 为x和y轴设置标签
axs[1].set_xlabel('x-coord (km)')
axs[1].set_ylabel('y-coord (km)')


# 这会改变子图的间距
fig.tight_layout()

# 展示绘图
plt.show()

左图显示了我们简单的一维螺位错模型。右图显示了该模型扩展到2D，假设我们的断层从东向西。在这种构造中，平行断层速度也是东西向速度。

虽然这些与断层平行的速度是我们想要的建模断层，但它们不是我们直接用InSAR测量的。SAR卫星如Sentinel-1以入射角$\theta$向下看地面，大约为31-46&deg;。这个角度在我们的场景中不断变化。InSAR也只能测量朝向和远离卫星的位移。这两个因素的结果是，InSAR只测量观测到的投影到卫星视线(LOS)的变形的一个分量，单一的视向无法区分不同的位移分量。

<img src="figures/incAngle.png" style="float: centre;" width="300"/>

*Figure 4: 卫星入射角$\theta$的示意图，在垂直(黑线虚线)和卫星视距(蓝色箭头)之间测量.*

让我们将视距投影应用于我们的信号，看看这对速度有什么影响。

interseismic_lib包含一个函数，可以根据卫星的方向旋转，在最小值和最大值之间创建一个入射角网格。记住Sentinel-1卫星是侧视的，因此我们的入射角将在90°变化;转到卫星方向。

In [ ]:
#航向(升轨-14，降轨-166)，最小入射角，最大入射角
#####################################
heading = -14 # degrees
inc_min, inc_max = 31, 46 # degrees
#####################################

# 创建这些角度的网格
heading_grid = np.ones(v_grid.shape) * heading
inc_grid = lib.gen_inc(inc_min, inc_max, heading, x, x)

# 从东西向los的投影
e2los = np.cos(np.deg2rad(heading_grid)) * np.sin(np.deg2rad(inc_grid))
v_grid_los = v_grid * e2los

# plot 
fig, axs = plt.subplots(2,2,figsize=(15,12))

# 为网格设置颜色限制
cmin, cmax = np.amin(v_grid*1000), np.amax(v_grid*1000)

# 入射角网格
im = axs[0,0].imshow(inc_grid, extent=[xmin, xmax, xmin, xmax], cmap=cm.batlow)
plt.colorbar(im, ax=axs[0,0], label="Incidence angle (degrees)")

# 东西向速度网格
im = axs[0,1].imshow(v_grid*1000, extent=[xmin, xmax, xmin, xmax], cmap=cm.vik, vmin=cmin, vmax=cmax)
plt.colorbar(im, ax=axs[0,1], label="East-west velocity (mm/yr)")
axs[0,1].plot([xmin, xmax], [0, 0], color='black', label='Fault trace') 
axs[0,1].plot([0, 0],[xmin, xmax], color='black', linestyle='dashed', label="Profile line")
axs[0,1].legend(fontsize=12)

# 1D models
axs[1,0].plot(x, v*1000, color='red', label='original (East-West) velocity')
axs[1,0].plot(x, v_grid_los[:,np.argmin(np.abs(x))]*1000, color='blue', label='projected LOS velocity')
axs[1,0].plot([0, 0],[axs[1,0].get_ylim()[0], axs[1,0].get_ylim()[1]], color='grey', linestyle='dashed')
axs[1,0].plot([axs[1,0].get_xlim()[0], axs[1,0].get_xlim()[1]], [0, 0], color='grey', linestyle='dashed')
axs[1,0].set_xlabel('Fault-perpendicular distance (km)')
axs[1,0].set_ylabel('Velocity (mm/yr)')
axs[1,0].legend()

# Line-of-sight velocity grid
im = axs[1,1].imshow(v_grid_los*1000, extent=[xmin, xmax, xmin, xmax], cmap=cm.vik, vmin=cmin, vmax=cmax)
plt.colorbar(im, ax=axs[1,1], label="Line-of-sight velocity (mm/yr)")
axs[1,1].plot([xmin, xmax], [0, 0], color='black')
axs[1,1].plot([0, 0],[xmin, xmax], color='black', linestyle='dashed')

plt.tight_layout()
plt.show()

我们可以看到，不仅整体信号变得更小，而且由于卫星的入射角不同，整个场景的信号也会发生变化。

现在我们已经探索了螺位错模型在InSAR中的表现，我们将继续将该模型应用于真实世界的数据。

## 2. 加载时间序列数据

W我们将从加载我们的LiCSBAS输出开始(您在LiCSBAS实践中生成了等效的输出)。为了清晰起见，它们被重命名为包含框架名称。变量名中的_asc和_desc标记表示升序帧或降序帧。

**vel_file** - 包含LOS速度的文件 (found in TS_GEOCml10/results/vel).

**par_file** - 参数文件，包含数据的地理位置，以及x和y中的大小 (found in GEOCml##).

**E/N/U_file** - 单位矢量分量，它基本上告诉我们如何从卫星LOS到东、北和上。稍后我们需要这些数据来进行速度分解。

In [ ]:
# setup file names
asc = '087A_04904_121313'
desc = '167D_04884_131212'

vel_file_asc = Path('data/' + asc + '.vel.mskd')
par_file_asc = Path('data/' + asc + '.par')
E_file_asc = Path('data/' + asc + '.E')
N_file_asc = Path('data/' + asc + '.N')
U_file_asc = Path('data/' + asc + '.U')

vel_file_desc = Path('data/' + desc + '.vel.mskd')
par_file_desc = Path('data/' + desc + '.par')
E_file_desc = Path('data/' + desc + '.E')
N_file_desc = Path('data/' + desc + '.N')
U_file_desc = Path('data/' + desc + '.U')

#读取数组维度 from par file
width_asc = int(lib.get_par(par_file_asc,'width'))
length_asc = int(lib.get_par(par_file_asc,'nlines'))

width_desc = int(lib.get_par(par_file_desc,'width'))
length_desc = int(lib.get_par(par_file_desc,'nlines'))

# get corner positions
corner_lat_asc = float(lib.get_par(par_file_asc,'corner_lat'))
corner_lon_asc = float(lib.get_par(par_file_asc,'corner_lon'))

corner_lat_desc = float(lib.get_par(par_file_desc,'corner_lat'))
corner_lon_desc = float(lib.get_par(par_file_desc,'corner_lon'))

# get post spacing (distance between velocity measurements)像素步长
post_lat_asc = float(lib.get_par(par_file_asc,'post_lat'))
post_lon_asc = float(lib.get_par(par_file_asc,'post_lon'))

post_lat_desc = float(lib.get_par(par_file_desc,'post_lat'))
post_lon_desc = float(lib.get_par(par_file_desc,'post_lon'))

# calculate grid spacings计算栅格空间大小
lat_asc = corner_lat_asc + post_lat_asc*np.arange(1,length_asc+1) - post_lat_asc/2
lon_asc = corner_lon_asc + post_lon_asc*np.arange(1,width_asc+1) - post_lon_asc/2

lat_desc = corner_lat_desc + post_lat_desc*np.arange(1,length_desc+1) - post_lat_desc/2
lon_desc = corner_lon_desc + post_lon_desc*np.arange(1,width_desc+1) - post_lon_desc/2

# load in velocities读取vel数据
vel_asc = np.fromfile(vel_file_asc, dtype='float32').reshape((length_asc, width_asc))
vel_desc = np.fromfile(vel_file_desc, dtype='float32').reshape((length_desc, width_desc))

# load in unit vectors加载分量信息
E_asc = np.fromfile(E_file_asc, dtype='float32').reshape((length_asc, width_asc))
N_asc = np.fromfile(N_file_asc, dtype='float32').reshape((length_asc, width_asc))
U_asc = np.fromfile(U_file_asc, dtype='float32').reshape((length_asc, width_asc))

E_desc = np.fromfile(E_file_desc, dtype='float32').reshape((length_desc, width_desc))
N_desc = np.fromfile(N_file_desc, dtype='float32').reshape((length_desc, width_desc))
U_desc = np.fromfile(U_file_desc, dtype='float32').reshape((length_desc, width_desc))

# load the frame polygons for plotting加载fram边缘范围的多边形数据
poly_asc = np.loadtxt(Path('data/' + asc + '.poly'))
poly_desc = np.loadtxt(Path('data/' + desc + '.poly'))

# load the naf fault trace加载断层痕迹数据
fault_trace = np.loadtxt(Path('data/naf_trace.xy'))

print('Files loaded.')
print('Size of ascending velocity:')
print(vel_asc.shape)
print('Size of descending velocity:')
print(vel_desc.shape)

print('Limits of ascending (lon, lat):')
print(np.amin(lon_asc), np.amax(lon_asc), np.amin(lat_asc), np.amax(lat_asc))
print('Limits of descending (lon, lat):')
print(np.amin(lon_desc), np.amax(lon_desc), np.amin(lat_desc), np.amax(lat_desc))

接下来，我们将绘制速度，以确保它们被正确加载。NAF的轨迹将被绘制为一条红线。

In [ ]:
# plot input vels
fig, axs = plt.subplots(1,2,figsize=(15,5))

# plot the ascending velocities, saturating the colour palette from -20 to 20 mm/yr.绘制升轨速率图
im = axs[0].imshow(vel_asc, extent=[np.amin(lon_asc), np.amax(lon_asc), np.amin(lat_asc), np.amax(lat_asc)], \
                   cmap=cm.vik, vmin=-20, vmax=20)

# add the NAF fault trace添加断层轨迹
axs[0].plot(fault_trace[:,0], fault_trace[:,1], color="red")

# add the frame polygon添加frame轮廓形状
axs[0].plot(poly_asc[:,0], poly_asc[:,1], color="black", linestyle='dashed')

# display the colorbar, this "divider" section is to scale the colorbar size to the plot显示颜色条，这个“divider”部分用于将颜色条的大小缩放到绘图中
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='LOS velocity (mm/yr)')

# set the title
axs[0].set_title('087A_04904_121313')

# set the x and y limits of the plot设置坐标轴刻度范围
axs[0].set_xlim(np.amin(lon_asc), np.amax(lon_asc))
axs[0].set_ylim(np.amin(lat_asc), np.amax(lat_asc))

# label the x and y axes
axs[0].set_xlabel('Longitude (degrees)')
axs[0].set_ylabel('Latitude (degrees)')

# repeat for the descending velocities重复步骤绘制降轨数据
im = axs[1].imshow(vel_desc, extent=[np.amin(lon_desc), np.amax(lon_desc), np.amin(lat_desc), np.amax(lat_desc)], \
                   cmap=cm.vik, vmin=-20, vmax=20)
axs[1].plot(fault_trace[:,0], fault_trace[:,1], color="red")
axs[1].plot(poly_desc[:,0], poly_desc[:,1], color="black", linestyle='dashed')
divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='LOS velocity (mm/yr)')
axs[1].set_title('167D_04884_131212')
axs[1].set_xlim(np.amin(lon_desc), np.amax(lon_desc))
axs[1].set_ylim(np.amin(lat_desc), np.amax(lat_desc))
axs[1].set_xlabel('Longitude (degrees)')
axs[1].set_ylabel('Latitude (degrees)')

# show the plot
fig.tight_layout()
plt.show()

默认情况下，LiCSBAS将朝向卫星的运动设置为正，将远离卫星的运动设置为负。这些速度相对于每一帧中licsbas选择的参考点。

我们已经可以在两张地图上看到断层上LOS速度的变化。为了进一步研究，让我们以升轨的速度绘制断层剖面。

In [ ]:
# start and end coordinates of the profile, and extra parameters剖面的开始和结束坐标，以及额外的参数
#####################################
prof_start = (33.5, 40.2)
prof_end = (33.1, 41.8)
prof_params = {
    "nbins": 100, # number of bins to split the profile into
    "width": 0.05 # total width of the profile in degrees (1 degree ~ 110 km)剖面总宽度(以度计)(1度~ 110公里)
}
#####################################

# run the profiler, the outputs are as follows:
# - bin_val = mean value of each bin
# - prof_bin_mids = distance along the profile to the middle of each bin
# - points_val = every velocity within the profile
# - points_dist = distance along the profile to every point within the profile
# - points_poly = polygon that defines the profile
运行剖面制作函数，输出如下:
# - bin_val =每个bin的平均值
# - prof_bin_mids =沿剖面到每个bin中间的距离*******
# - points_val =剖面范围内的所有的速度数据（所包含的所有栅格数据点）
# - points_dist =每个点到剖面的距离******
# - points_poly =剖面形状的多边形轮廓数据
bin_val, prof_bin_mids, points_val, points_dist, points_poly \
    = lib.profile_data(lon_asc,lat_asc,vel_asc,prof_start,prof_end,prof_params)

# calculate profile-fault intersection angle计算剖面-断层交会角
intersect_dist, intersect_angle = lib.profile_fault_intersection(prof_start,prof_end,fault_trace)

# plot the ascending velocities and the profiled data绘制升轨速度和剖面数据
fig, axs = plt.subplots(1,2,figsize=(15,5))

# ascending velocity map与之前相同的绘制升轨数据
im = axs[0].imshow(vel_asc, extent=[np.amin(lon_asc), np.amax(lon_asc), np.amin(lat_asc), np.amax(lat_asc)], \
                   cmap=cm.vik, vmin=-20, vmax=20)
axs[0].plot(fault_trace[:,0], fault_trace[:,1], color="red")
axs[0].plot([prof_start[0], prof_end[0]], [prof_start[1], prof_end[1]], color="red")
axs[0].plot(points_poly[:,0],points_poly[:,1], color="red")
axs[0].scatter(prof_start[0],prof_start[1], s=100, color='pink', edgecolor='black', zorder=3)
axs[0].scatter(prof_end[0],prof_end[1], s=100, color='black', edgecolor='white', zorder=3)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='LOS velocity (mm/yr)')
axs[0].set_title('087A_04904_121313')
axs[0].set_xlim(np.amin(lon_asc), np.amax(lon_asc))
axs[0].set_ylim(np.amin(lat_asc), np.amax(lat_asc))

# velocities projected onto profile投射到剖面上的速度
axs[1].scatter(points_dist, points_val, s=2)
axs[1].plot([intersect_dist, intersect_dist],[axs[1].get_ylim()[0], axs[1].get_ylim()[1]], color='grey', linestyle='dashed')
axs[1].plot([axs[1].get_xlim()[0], axs[1].get_xlim()[1]], [0, 0], color='grey', linestyle='dashed')
axs[1].set_xlabel("Distance along profile (degrees)")
axs[1].set_ylabel("Line-of-sight velocity (mm/yr)")

fig.tight_layout()
plt.show()

左图显示了我们的速度图。粉色圆点表示剖面的开始，黑色圆点表示剖面的结束。它们之间的三条红线表示主剖面线和剖面宽度的范围。

我们的剖面速度显示在右边。这些都被投影到剖面线上，因此从prof_start沿着剖面线的距离给出，prof_start位于0距离处。距离的单位与剖面数据集的单位相同。垂直虚线表示剖面线与NAF的断层轨迹相交的地方。
虽然剖面上的速度有一点噪声，但你应该能够看到断层上速度的总体变化。

## 3. 将视线速度分解为断层平行速度和垂直速度

方程1需要断层平行速度，而我们的速度目前是卫星LOS向速度。
我们可以通过将LOS速度分解为断层平行速度、断层垂直速度和垂直速度来估计断层平行速度。（分解为三个方向速度）
这是通过求解InSAR速度($V_{asc}$和$V_{desc}$)与分量速度之间的线性方程组来完成的。
我们使用雷达入射角$\theta$和沿轨道卫星航向的方位角$\alpha$以及我们的断层走向$\gamma$在两者之间进行转换。$\theta$和$\alpha$每个像素都不同。

首先，让我们看看如何将LOS速度分解为东($V_E$)，北($V_N$)和垂直($V_U$)速度:
\begin{equation}
    \begin{bmatrix} V_{asc} \\ V_{desc} \end{bmatrix} = \begin{bmatrix} sin(\theta_{asc})cos(\alpha_{asc}) & sin(\theta_{asc})sin(\alpha_{asc}) & -cos(\theta_{asc}) \\ sin(\theta_{desc})cos(\alpha_{desc}) & sin(\theta_{desc})sin(\alpha_{desc}) & -cos(\theta_{desc}) \end{bmatrix} \begin{bmatrix} V_E \\ V_N \\ V_U \end{bmatrix}
    \qquad \qquad (2)
\end{equation}

我们将断层平行速度($V_{para}$)和断层垂直速度($V_{perp}$)计算为$V_E$和$V_N$的相关分量之和，其中包括一个额外的项，用于说明从北顺时针测量的断层走向: 


\begin{equation}
    \begin{bmatrix} V_{asc} \\ V_{desc} \end{bmatrix} = \begin{bmatrix} \begin{pmatrix} sin(\gamma)sin(\theta_{asc})cos(\alpha_{asc})\\+ cos(\gamma)sin(\theta_{asc})sin(\alpha_{asc}) \end{pmatrix} & \begin{pmatrix} cos(\gamma)sin(\theta_{asc})cos(\alpha_{asc})\\+ sin(\gamma)sin(\theta_{asc})sin(\alpha_{asc}) \end{pmatrix} & -cos(\theta_{asc}) \\ \begin{pmatrix} sin(\gamma)sin(\theta_{desc})cos(\alpha_{desc})\\+ cos(\gamma)sin(\theta_{desc})sin(\alpha_{desc}) \end{pmatrix} & \begin{pmatrix} cos(\gamma)sin(\theta_{desc})cos(\alpha_{desc})\\+ sin(\gamma)sin(\theta_{desc})sin(\alpha_{desc}) \end{pmatrix} & -cos(\theta_{desc}) \end{bmatrix} \begin{bmatrix} V_{para} \\ V_{perp} \\ V_U \end{bmatrix}
    \qquad \qquad (3)
\end{equation}

在这两种情况下，我们都有三个未知数和两个数据集，使其成为一个欠确定系统。
假设我们可用的数据量是固定的，我们需要将我们想要估计的分量速度的数量减少一个，这样我们就可以解出剩下的两个。
我们假设$V_{perp}$为零，这样我们就可以解出$V_{para}$和$V_U$。我们选择$V_{perp}$是因为我们预计主要信号是东西走向的NAF上的走滑运动，产生一个小的南北分量，哨兵-1卫星由于其近极轨道而对其特别不敏感。

现在我们可以重写方程3:

\begin{equation}
    \begin{bmatrix} V_{asc} \\ V_{desc} \end{bmatrix} = \begin{bmatrix} sin(\gamma)sin(\theta_{asc})cos(\alpha_{asc})+cos(\gamma)sin(\theta_{asc})sin(\alpha_{asc}) & -cos(\theta_{asc}) \\ sin(\gamma)sin(\theta_{desc})cos(\alpha_{desc})+cos(\gamma)sin(\theta_{desc})sin(\alpha_{desc}) & -cos(\theta_{desc}) \end{bmatrix} \begin{bmatrix} V_{para} \\ V_U \end{bmatrix}
    \qquad \qquad (4)
\end{equation}

式中$V_{asc}$、$V_{desc}$为目视速度，$\theta$为雷达入射角，$\alpha$为卫星沿轨航向方位角。从LOS到故障并行的转换可以简化为$sin(\theta)sin(\alpha + \gamma)$，但是，我们只有可用的单位分量向量，而不是$\theta$和$\alpha$的单个值，因此我们将坚持使用公式4。    

目前，这些速度在两个稍有不同的网格上。我们需要统一网格，这样我们才能对共享点进行速度分解。

In [ ]:
# 根据升轨和降轨的栅格数据范围，新建一个能够包含全部的新的栅格数据范围图（升轨和降轨各自制作一个），将升轨和降轨数据重新填入
# limits and intervals for new grid新网格的限制和间隔
lon_min = np.amin([lon_asc[0], lon_desc[0]])
lon_max = np.amax([lon_asc[-1], lon_desc[-1]])
lon_int = np.amin([post_lon_asc, post_lon_desc])
lon_regrid = np.arange(lon_min,lon_max+lon_int,lon_int)

lat_max = np.amax([lat_asc[0], lat_desc[0]])
lat_min = np.amin([lat_asc[-1], lat_desc[-1]])
lat_int = np.absolute(np.amin([post_lat_asc, post_lat_desc]))
lat_regrid = np.arange(lat_min,lat_max+lat_int,lat_int)

xx_regrid, yy_regrid = np.meshgrid(lon_regrid, lat_regrid[::-1])
coords_regrid = np.transpose(np.vstack((xx_regrid.flatten(),yy_regrid.flatten())))

# interpolate velocities onto this new grid将速度插值到这个新网格上
xx_asc, yy_asc = np.meshgrid(lon_asc, lat_asc)
vel_asc_regrid = interpolate.griddata((xx_asc.ravel(), yy_asc.ravel()), vel_asc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel()))
vel_asc_regrid = vel_asc_regrid.reshape((len(lat_regrid),len(lon_regrid)))

xx_desc, yy_desc = np.meshgrid(lon_desc, lat_desc)
vel_desc_regrid = interpolate.griddata((xx_desc.ravel(), yy_desc.ravel()), vel_desc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel()))
vel_desc_regrid = vel_desc_regrid.reshape((len(lat_regrid),len(lon_regrid)))

# interpolate the component vectors into the new grid将分量向量插入到新的网格中
E_asc_regrid = interpolate.griddata((xx_asc.ravel(), yy_asc.ravel()), E_asc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))
N_asc_regrid = interpolate.griddata((xx_asc.ravel(), yy_asc.ravel()), N_asc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))
U_asc_regrid = interpolate.griddata((xx_asc.ravel(), yy_asc.ravel()), U_asc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))

E_desc_regrid = interpolate.griddata((xx_desc.ravel(), yy_desc.ravel()), E_desc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))
N_desc_regrid = interpolate.griddata((xx_desc.ravel(), yy_desc.ravel()), N_desc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))
U_desc_regrid = interpolate.griddata((xx_desc.ravel(), yy_desc.ravel()), U_desc.ravel(), (xx_regrid.ravel(), yy_regrid.ravel())).reshape((len(lat_regrid),len(lon_regrid)))

print('Regridding complete.')
print('Size of ascending velocity:')
print(vel_asc_regrid.shape)
print('Size of descending velocity:')
print(vel_desc_regrid.shape)

目前，这两个速度场相对于它们自己的参考像素。我们需要使这个参考相同，这样速度就可以被正确地分解。我们将定义一个参考窗口，并使用该区域内所有速度的平均值作为参考。（此参考窗口在升降轨共同范围内，且完全相同）

In [ ]:
# define new reference pixel
ref_xmin, ref_xmax = 32.65, 32.85
ref_ymin, ref_ymax = 41.1, 41.2

# ref poly for plotting
ref_poly = np.array([[ref_xmin, ref_ymin],
           [ref_xmin, ref_ymax],
           [ref_xmax, ref_ymax],
           [ref_xmax, ref_ymin],
           [ref_xmin, ref_ymin]])

# get index
ind_xmin = np.argmin(np.absolute(lon_regrid-ref_xmin))
ind_xmax = np.argmin(np.absolute(lon_regrid-ref_xmax))
ind_ymin = np.argmin(np.absolute(lat_regrid-ref_ymin))
ind_ymax = np.argmin(np.absolute(lat_regrid-ref_ymax))

# get ref value
ref_val_asc = np.nanmean(vel_asc_regrid[ind_ymin:ind_ymax+1,ind_xmin:ind_xmax+1])
ref_val_desc = np.nanmean(vel_desc_regrid[ind_ymin:ind_ymax+1,ind_xmin:ind_xmax+1])

# set as new reference
vel_asc_regrid = vel_asc_regrid - ref_val_asc
vel_desc_regrid = vel_desc_regrid - ref_val_desc

print('Referencing complete.')
print('Mean value of reference for ascending = ' + str(round(ref_val_asc,3)) + ' mm/yr')
print('Mean value of reference for descending = ' + str(round(ref_val_desc,3)) + ' mm/yr')

In [ ]:
# 画出重新计算的速度，确保它们是正确的
fig, axs = plt.subplots(1,2,figsize=(15,10))

# ascending velocities
im = axs[0].imshow(vel_asc_regrid, extent=[lon_regrid[0], lon_regrid[-1], lat_regrid[0], lat_regrid[-1]], \
                   cmap=cm.vik, vmin=-20, vmax=20)
axs[0].plot(fault_trace[:,0], fault_trace[:,1], color="red")
axs[0].plot(ref_poly[:,0], ref_poly[:,1], color='black')
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='LOS velocity (mm/yr)')
axs[0].set_title('087A_04904_121313')
axs[0].set_xlim(np.amin(lon_regrid), np.amax(lon_regrid))
axs[0].set_ylim(np.amin(lat_regrid), np.amax(lat_regrid))

# descending velocities
im = axs[1].imshow(vel_desc_regrid, extent=[lon_regrid[0], lon_regrid[-1], lat_regrid[0], lat_regrid[-1]], \
                   cmap=cm.vik, vmin=-20, vmax=20)
axs[1].plot(fault_trace[:,0], fault_trace[:,1], color="red")
axs[1].plot(ref_poly[:,0], ref_poly[:,1], color='black')
divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='LOS velocity (mm/yr)')
axs[1].set_title('167D_04884_131212')
axs[1].set_xlim(np.amin(lon_regrid), np.amax(lon_regrid))
axs[1].set_ylim(np.amin(lat_regrid), np.amax(lat_regrid))

fig.tight_layout(w_pad=3)
plt.show()

Now we'll run the decomposition for each pixel, assuming no correlation between pixels and a strike of 80°.

现在我们将对每个像素进行分解，假设像素之间不存在相关性并且断层走向为 80°。

In [ ]:
# set strike
strike = 80
strike_E, strike_N = np.sin(np.deg2rad(strike)), np.cos(np.deg2rad(strike))

# pre-allocate
vel_para = np.zeros((len(lat_regrid), len(lon_regrid)))
vel_U = np.zeros((len(lat_regrid), len(lon_regrid)))

# loop through every pixel
for ii in np.arange(0,len(lat_regrid)):
    for jj in np.arange(0,len(lon_regrid)):
        
        # create the design matrix
        G = np.array([[strike_E*E_asc_regrid[ii,jj] + strike_N*N_asc_regrid[ii,jj], U_asc_regrid[ii,jj]], 
                      [strike_E*E_desc_regrid[ii,jj] + strike_N*N_desc_regrid[ii,jj], U_desc_regrid[ii,jj]]])
        
        # get the two velocities for this pixel
        d = np.array([[vel_asc_regrid[ii,jj], vel_desc_regrid[ii,jj]]]).T
        
        # solve the linear system for the Up and East velocities
        m = np.linalg.solve(G, d)
        
        # save to arrays
        vel_para[ii,jj] = m[0]
        vel_U[ii,jj] = m[1]
        
print('Velocity decomposition complete.')

Plot the decomposed velocities, including the frame polygons so that we can see how we only get the decomposed velocities in the overlap area, where we have both line-of-sights to work with.

绘制分解的速度，包括frame边框，这样我们就可以看到我们如何只在重叠区域得到分解的速度，在那里我们有两个los来工作。

In [ ]:
# plot the results
fig, axs = plt.subplots(1,2,figsize=(15,10))

# East velocities
im = axs[0].imshow(vel_para, extent=[lon_regrid[0], lon_regrid[-1], lat_regrid[0], lat_regrid[-1]], \
                   cmap=cm.vik, vmin=-20, vmax=20)
axs[0].plot(fault_trace[:,0], fault_trace[:,1], color="red")
axs[0].plot(poly_asc[:,0], poly_asc[:,1], color="black", linestyle='dashed')
axs[0].plot(poly_desc[:,0], poly_desc[:,1], color="black", linestyle='dashed')
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='Fault-parallel velocity (mm/yr)')
axs[0].set_title('Fault-parallel')
axs[0].set_xlim(np.amin(lon_regrid), np.amax(lon_regrid))
axs[0].set_ylim(np.amin(lat_regrid), np.amax(lat_regrid))

# Up velocities
im = axs[1].imshow(vel_U, extent=[lon_regrid[0], lon_regrid[-1], lat_regrid[0], lat_regrid[-1]], \
                   cmap=cm.vik, vmin=-20, vmax=20)
axs[1].plot(fault_trace[:,0], fault_trace[:,1], color="red")
axs[1].plot(poly_asc[:,0], poly_asc[:,1], color="black", linestyle='dashed')
axs[1].plot(poly_desc[:,0], poly_desc[:,1], color="black", linestyle='dashed')
divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='Vertical velocity (mm/yr)')
axs[1].set_title('Up')
axs[1].set_xlim(np.amin(lon_regrid), np.amax(lon_regrid))
axs[1].set_ylim(np.amin(lat_regrid), np.amax(lat_regrid))

plt.tight_layout(w_pad=3)
plt.show()

## 4. Profile the fault-parallel velocities断层平行速度的剖面图

We'll now profile our decomposed fault-parallel velocities to produce a dataset which we can then apply our screw dislocation model to. First, we need to project our lat-long coordinates into Universal Transverse Mercator (UTM) so that the profile distances are in metres and not degrees. After this projection, our velocities will no longer be on a regular grid, so we'll plot them using scatter instead of imshow.

现在，我们将对分解的断层平行速度进行分析，以产生一个数据集，然后我们可以将螺旋位错模型应用于该数据集。首先，我们需要将我们的经纬度坐标投影到通用横向墨卡托(UTM)中，以便剖面距离以米为单位而不是以度为单位。在这个投影之后，我们的速度将不再在一个规则的网格上，所以我们将使用散点而不是imshow来绘制它们。

In [ ]:
# Convert to UTM using gdal 投影转换

# get the utm zone projection
utm_crs_list = pyproj.database.query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest = pyproj.aoi.AreaOfInterest(
        west_lon_degree  = np.amin(lon_regrid),
        south_lat_degree = np.amin(lat_regrid),
        east_lon_degree  = np.amax(lon_regrid),
        north_lat_degree = np.amax(lat_regrid),
    ),
)
utm_crs = pyproj.CRS.from_epsg(utm_crs_list[0].code)

# create transformer for LL to UTM 
transformer = pyproj.Transformer.from_crs('epsg:4326', utm_crs)

# apply transform to our grids of lat long coordinates
xx_utm, yy_utm = transformer.transform(yy_regrid, xx_regrid)

# apply transform to fault trace
fault_trace_utm = fault_trace.copy()
fault_trace_utm[:,0], fault_trace_utm[:,1] = transformer.transform(fault_trace[:,1], fault_trace[:,0])

# convert utm metres to km (easier to work with for plotting)
xx_utm = xx_utm / 1000
yy_utm = yy_utm / 1000
fault_trace_utm = fault_trace_utm / 1000

# get new limits for plotting
val_ind_row, val_ind_col = np.where(~np.isnan(vel_U).all(axis=0))[0], np.where(~np.isnan(vel_U).all(axis=1))[0]
xlim = [xx_utm[-1,val_ind_row[0]], xx_utm[-1,val_ind_row[-1]]]
ylim = [yy_utm[val_ind_col[-1],0], yy_utm[val_ind_col[0],0]]

# replot with new coordinates
fig, axs = plt.subplots(1,2,figsize=(15,7))

# East velocities
im = axs[0].scatter(xx_utm.flatten(), yy_utm.flatten(), s=2, c=vel_para.flatten(), cmap=cm.vik, vmin=-20, vmax=20)
axs[0].plot(fault_trace_utm[:,0], fault_trace_utm[:,1], color="red")
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='Fault-parallel velocity (mm/yr)')
axs[0].set_aspect('equal', 'box')
axs[0].set_title('Fault-parallel')
axs[0].set_xlabel('x-coord (km)')
axs[0].set_ylabel('y-coord (km)')
axs[0].set_xlim(xlim)
axs[0].set_ylim(ylim)

# Up velocities
im = axs[1].scatter(xx_utm.flatten(), yy_utm.flatten(), s=2, c=vel_U.flatten(), cmap=cm.vik, vmin=-20, vmax=20)
axs[1].plot(fault_trace_utm[:,0], fault_trace_utm[:,1], color="red")
divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='Vertical velocity (mm/yr)')
axs[1].set_aspect('equal', 'box')
axs[1].set_title('Up')
axs[1].set_xlabel('x-coord (km)')
axs[1].set_ylabel('y-coord (km)')
axs[1].set_xlim(xlim)
axs[1].set_ylim(ylim)

fig.tight_layout(w_pad=5)
plt.show()

In [ ]:
# Set the profile parameters

# rough middle profile
#####################################
prof_start = (573, 4450)
prof_end = (540, 4595)
prof_params = {
    "nbins": 100, # number of bins to split the profile into
    "width": 5  # total width of the profile in km
}
#####################################

# run the profiler
bin_val, prof_bin_mids, points_val, points_dist, points_poly \
    = lib.profile_data(xx_utm,yy_utm,vel_para,prof_start,prof_end,prof_params)

# calculate profile-fault intersection angle
intersect_dist, intersect_angle = lib.profile_fault_intersection(prof_start,prof_end,fault_trace_utm)

# plot
fig, axs = plt.subplots(1,2,figsize=(15,5))

# East velocities
im = axs[0].scatter(xx_utm.flatten(), yy_utm.flatten(), s=2, c=vel_para.flatten(), cmap=cm.vik, vmin=-20, vmax=20)
axs[0].plot(fault_trace_utm[:,0], fault_trace_utm[:,1], color="red")
axs[0].plot([prof_start[0], prof_end[0]], [prof_start[1], prof_end[1]], color="red")
axs[0].plot(points_poly[:,0],points_poly[:,1], color="red")
axs[0].scatter(prof_start[0],prof_start[1], s=100, color='pink', edgecolor='black', zorder=3)
axs[0].scatter(prof_end[0],prof_end[1], s=100, color='black', edgecolor='white', zorder=3)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='Fault-parallel velocity (mm/yr)')
axs[0].set_aspect('equal', 'box')
axs[0].set_title('Fault-parallel')
axs[0].set_xlabel('x-coord (km)')
axs[0].set_ylabel('y-coord (km)')
axs[0].set_xlim(xlim)
axs[0].set_ylim(ylim)

# profile
axs[1].scatter(points_dist,points_val)
axs[1].plot(prof_bin_mids,bin_val,color="red")
axs[1].plot([intersect_dist, intersect_dist],[axs[1].get_ylim()[0], axs[1].get_ylim()[1]], color='grey', linestyle='dashed')
axs[1].plot([axs[1].get_xlim()[0], axs[1].get_xlim()[1]], [0, 0], color='grey', linestyle='dashed')
axs[1].text(0.02, 0.90, 'intersection angle = ' + str(round(np.abs(intersect_angle))) + ' degrees', fontsize=14, transform = axs[1].transAxes)
axs[1].set_xlabel("Distance along profile (km)")
axs[1].set_ylabel("Fault-parallel velocity (mm/yr)")
axs[1].yaxis.set_label_position("right")
axs[1].yaxis.tick_right()

fig.tight_layout(w_pad=4)
plt.show()

I've included the intersection angle between the fault and the profile line on the right-hand plot.
We want this angle to be 90&deg;, otherwise the signal will become smeared and effect our results.

我在右边的图中包含了断层和剖面线的交汇角。
我们希望这个角度是90度，否则信号会变得模糊并影响我们的结果。

## 5. Forward model 拟合模型

Now that we have our data, let's produce a simple forward model to get an idea of what model parameters produce a reasonable fit.

现在我们有了数据，让我们生成一个简单的之前提到的模型，以了解什么样的模型参数产生什么样的拟合结果。

In [ ]:
# Slip, locking depth, and offset
#####################################
s = 15 # mm/yr
d = 15 # km
c = -5 # mm/yr
#####################################

# shift profile so that the fault is at 0 km
x_prof = points_dist - intersect_dist

# Vector of positions every 1 km for -100 km to 100 km
x = np.arange(-100,101,1)

# run the forward model
v = lib.screw_disc(x*1000, s/1000, d*1000, c/1000)

# calculate rms misfit by running the forward model for x positions with velocities
v_forward = lib.screw_disc(x_prof*1000, s/1000, d*1000, c/1000)
rms_forward = lib.rms_misfit(points_val,v_forward*1000)

# Plot comparison
fig, axs = plt.subplots(1,1,figsize=(15,8))

plt.scatter(x_prof,points_val)
plt.plot(x, v*1000, c='r')
plt.plot([0, 0],[axs.get_ylim()[0], axs.get_ylim()[1]], color='grey', linestyle='dashed')
plt.plot([axs.get_xlim()[0], axs.get_xlim()[1]], [0, 0], color='grey', linestyle='dashed')

plt.text(0.15, 0.9, 'slip rate = ' + str(s) + ' mm/yr', fontsize=14, transform = axs.transAxes)
plt.text(0.15, 0.84, 'locking depth = ' + str(d) + ' km', fontsize=14, transform = axs.transAxes)
plt.text(0.15, 0.78, 'offset = ' + str(c) + ' mm/yr', fontsize=14, transform = axs.transAxes)
plt.text(0.15, 0.72, 'RMS misfit = ' + str(round(rms_forward,3)) + ' mm/yr', fontsize=14, transform = axs.transAxes)

plt.xlabel('Perpendicular distance from surface fault trace (km)')
plt.ylabel('Fault-parallel velocity (mm/yr)')

plt.show()

To see how well you can all model the fault, lets model the same profile and then share the results.
为了展示你对断层建模的好坏，让我们用一个简单的断层数据建模并分享结果

Set your profile to the following:设置断层如下：
```
prof_start = (573, 4450)
prof_end = (540, 4595)
```

Once you have your best model, let me know your parameter values.
当你获得了最佳模型结果就会知道这些参数值的意义

## 6. Bayesian inversion贝叶斯反演

Bayesian inversions are one way we can estimate model parameters and their uncertainties in geophysics. We take some prior knowledge about the model (e.g. limits of the model parameter values) and combine this with a likelihood function based on the observations to estimate the posterior probability (i.e. the likelihood of our model parameters being the true values given the data). By exploring a range of model parameter values, we can build up a a posterior probability distribution which tells us both our "best fit" values, known as the "maximum a posteriori probability (MAP)" solution, and allows us to estimate the uncertainties on this solution.

贝叶斯反演是地球物理学中估计模型参数及其不确定性的一种方法。我们采用一些关于模型的先验知识(例如，模型参数值的极限)，并将其与基于观察值的似然函数相结合，以估计后验概率(即，给定数据的模型参数是真实值的可能性)。通过探索模型参数值的范围，我们可以建立一个后验概率分布，该分布告诉我们“最佳拟合”值，即“最大后验概率(MAP)”解，并允许我们估计该解的不确定性。

We'll assume uniform priors for each model parameter, meaning all values between the upper and lower limits are equally likely. Values outside of these limits are "impossible" and will be rejected in our inversion.

我们将假设每个模型参数的先验一致，这意味着上限和下限之间的所有值都是等可能的。超出这些限制的值是“不可能的”，在我们的反演中将被拒绝。

We use the following limits for our uniform priors:我们对统一先验使用以下限制:
- $0 \leq s \leq 40$ mm/yr
- $1 \leq d \leq 50$ km
- $-10 \leq c \leq 10$ mm/yr

Assuming our errors are Gaussian, we can find the maximum likelihood by minimising the weighted difference between the observed velocities, $\mathbf d$, and the model velocities, $g(\mathbf{m})$:我们对统一先验使用以下限制:
\begin{equation}
    \sum (\mathbf d-g(\mathbf{m}))*W*(\mathbf d-g(\mathbf{m}))
\end{equation}
where $W$ contains the weight for every velocity. Weighting the data is useful when some velocities have larger errors than others. For simplicity, we'll assume a uniform uncertainty of 1 mm/yr (1 standard deviation) across all of our velocities.

其中$W$包含每个速度的重权重。当某些速度的误差大于其他速度时，对数据进行加权是有用的。为了简单起见，我们假设所有速度的不确定度都是1毫米/年(1个标准差)

In order to explore the posterior distribution (i.e. test a large number of model parameter values) we'll use a Markov Chain Monte Carlo (MCMC) method.
Our MCMC performs the following steps:
- Take a starting model and add a random step to each parameters.
- Test whether this new model is within the prior limits, rejecting the model if not.
- Calculate the likelihoods for both models.
- If the new model has a higher likelihood, then accept this model. Also accept a number of models that are worse to help the Monte Carlo explore the full parameter space.

为了探索后验分布(即测试大量模型参数值)，我们将使用马尔可夫链蒙特卡罗(MCMC)方法。
我们的MCMC执行以下步骤:

- 取一个初始模型，并为每个参数添加一个随机步骤。
- 测试此新模型是否在先前的限制范围内，如果不在，则拒绝该模型。
- 计算两种模型的可能性。
- 如果新的模型有更高的可能性，那么接受这个模型。还接受一些较差的模型，帮助蒙特卡罗探索全参数空间。

We need to allow the MCMC to sometimes accept less likely models to avoid local minima. These are models that are good compared to the surrounding models (in the parameter space), but are not the true best model (called the global minima).

我们需要允许MCMC有时接受不太可能的模型来避免局部最小值。这些模型与周围的模型(在参数空间中)相比是好的，但不是真正的最佳模型(称为全局最小值)。

In [ ]:
importlib.reload(lib)
np.random.seed(1)

# inversion setup
#####################################
n_iterations = 10000 # don't go above this value of 10,000 in the practical
#####################################

# take profiled velocities and shift the fault to be at zero
x_prof = points_dist - intersect_dist
v_prof = points_val

# model params and their limits
m_start = np.array([5, 5, 0], dtype=np.double) # slip in mm/yr, locking depth in km
m_min = np.array([0, 1, -10], dtype=np.double)
m_max = np.array([40, 50, 10], dtype=np.double)

# variance-covariance matrix, currently 1 mm/yr uncertainties
vcm = np.eye(len(x_prof))

# convert all units to metres
x = x_prof * 1000
v = v_prof / 1000
m_start = m_start * np.array([0.001, 1000, 0.001])
m_min = m_min * np.array([0.001, 1000, 0.001])
m_max = m_max * np.array([0.001, 1000, 0.001])
vcm = vcm / 1e6

# inversion setup
burn_in = round(n_iterations/100)
acceptance_coef = 0.001

# weightings
W = np.linalg.inv(vcm)

# pre-allocate arrays
models_saved=np.zeros((n_iterations,len(m_start))) * np.nan
ll_saved=np.zeros((n_iterations,1)) * np.nan
n_accept = 0 # number of accepted models
n_reject = 0 # number of rejected models

# pass starting model to current model
m_current = m_start.copy()

# run inversion
for ii in range(n_iterations):
    
    # propose model using different step sizes for each parameter
    m_trial = m_current.copy()
    m_trial[0] = m_trial[0] + np.random.normal(loc=0, scale=2.5, size=1)/1000 # slip rate
    m_trial[1] = m_trial[1] + np.random.normal(loc=0, scale=2.5, size=1)*1000 # locking depth
    m_trial[2] = m_trial[2] + np.random.normal(loc=0, scale=1, size=1)/1000 # offset
    
    # check limits and skip the rest of the loop if any parameter is invalid
    if not(lib.prior(m_trial,m_min,m_max)):
        n_reject += 1
        models_saved[ii,:] = m_current
        continue
    
    # calculate likelihood for the current and trial models
    ll_current = lib.likelihood(x, v, m_current, W)
    ll_trial = lib.likelihood(x, v, m_trial, W)
    
    # test whether to keep trial model
    #if np.exp(acceptance_coef*(ll_trial-ll_current)) > np.random.uniform(low=0, high=1,size=1):
    if np.exp(acceptance_coef*(ll_current-ll_trial)) > np.random.uniform(low=0, high=1,size=1):
        m_current = m_trial
        ll_current = ll_trial
        n_accept += 1
    else:
        n_reject += 1
    models_saved[ii,:] = m_current
    ll_saved[ii] = ll_current
    
# convert back from metres to mm/yr and km
models_saved[:,0] = models_saved[:,0] * 1000 # slip rate
models_saved[:,1] = models_saved[:,1] / 1000 # locking depth
models_saved[:,2] = models_saved[:,2] * 1000 # offset

# find best fit model using min of likelihood function
best_model = models_saved[np.nanargmin(ll_saved),:]

print('Inversion complete')
print('Number of accepted models = ' + str(n_accept))
print('Number of rejected models = ' + str(n_reject))

First, we'll plot the accepted trial models for the slip rate and locking depth to see how the algorithm explores the parameter space.
We'll show the first 1% of iterations in red and the rest in blue.
These early iterations are often termed a "burn-in" period, where the inversion is moving from the start position towards the (global) minimum, which it then begins to explore.

首先，我们将绘制出滑移率和锁定深度的可接受试验模型，以了解算法如何探索参数空间。
我们将用红色显示前1%的迭代，其余的用蓝色显示。
这些早期的迭代通常被称为“磨合”期，在此期间，反转从开始位置移动到(全局)最小值，然后开始探索最小值。

In [ ]:
# plot walker 
plt.figure()
plt.plot(models_saved[:burn_in,0], models_saved[:burn_in,1], color='red', alpha=0.3)
plt.plot(models_saved[burn_in:,0], models_saved[burn_in:,1], color='blue', alpha=0.3)
plt.xlabel('Slip rate (mm/yr)')
plt.ylabel('Locking depth (km)')

We can also look at how the inversion explored the parameter space by plotting the likelihood function value for each iteration. The lowest likelihood will be located within a minima, while the higher likelihoods throughout show the inversion accepting worse models to help it escape local minima.

我们还可以通过绘制每次迭代的似然函数值来查看反转如何探索参数空间。最小的似然值将位于一个极小值内，而较高的似然值表明，反演接受较差的模型，以帮助它逃避局部极小值。

In [ ]:
# plot likelihood value throughout monte carlo
fig, axs = plt.subplots(1,1,figsize=(15,8))

axs.scatter(np.arange(1,len(ll_saved)+1),ll_saved, s=3)
axs.set_xlabel('Number of iterations')
axs.set_ylabel('Weighted residual')

plt.show()

Next, lets plot our best model on top of the profiled velocities.接下来，让我们在速度曲线上绘制最佳模型。

In [ ]:
# calculate rms misfit
v_final = lib.screw_disc(x_prof, best_model[0], best_model[1], best_model[2])
best_rms = lib.rms_misfit(v_prof,v_final)

# for plotting
x = np.arange(-100,101,1)
v_final_plot = lib.screw_disc(x, best_model[0], best_model[1], best_model[2])

# Plot comparison
fig, axs = plt.subplots(1,1,figsize=(15,8))

plt.scatter(x_prof,v_prof)
plt.plot(x, v_final_plot, c='r')
plt.plot([0, 0],[axs.get_ylim()[0], axs.get_ylim()[1]], color='grey', linestyle='dashed')

plt.text(0.15, 0.8, 'slip = ' + str(round(best_model[0],2)) + ' mm/yr', fontsize=14, transform = axs.transAxes)
plt.text(0.15, 0.74, 'locking depth = ' + str(round(best_model[1],2)) + ' km', fontsize=14, transform = axs.transAxes)
plt.text(0.15, 0.68, 'offset = ' + str(round(best_model[2],2)) + ' mm/yr', fontsize=14, transform = axs.transAxes)
plt.text(0.15, 0.62, 'RMS misfit = ' + str(round(best_rms,3)) + ' mm/yr', fontsize=14, transform = axs.transAxes)

plt.xlabel('Perp. distance from fault (km)')
plt.ylabel('Fault-parallel velocity (mm/yr)')

plt.show()

Now we'll plot histograms for each model parameter. The overall shape of these histograms should be gaussian, although they may be skewed to one side. They may also be quite rough depending on the number of iterations that have been run. By fitting a gaussian to each histogram, we can estimate the standard deviation of each, which is a measure of the uncertainty on our estimates.

We'll plot the best model values as vertical red lines. These should roughly line up with the peaks of our histograms.

现在我们将绘制每个模型参数的直方图。这些直方图的总体形状应该是高斯的，尽管它们可能向一侧倾斜。根据所运行的迭代次数，它们也可能相当粗糙。通过对每个直方图拟合高斯分布，我们可以估计每个直方图的标准差，这是我们估计的不确定性的度量。

我们将把最佳模型值绘制为垂直的红线。这些应该与我们的直方图的峰值大致一致。

In [ ]:
# calculate mean and standard deviation for each parameter
gauss_s = stats.norm.fit(models_saved[burn_in:,0])
gauss_d = stats.norm.fit(models_saved[burn_in:,1])
gauss_c = stats.norm.fit(models_saved[burn_in:,2])

# create figure
fig, axs = plt.subplots(1, 3, figsize=(15,8))

# plot histograms
_, bins_s, _ = axs[0].hist(models_saved[burn_in:,0], bins=20, density=True)
_, bins_d, _ = axs[1].hist(models_saved[burn_in:,1], bins=20, density=True)
_, bins_c, _ = axs[2].hist(models_saved[burn_in:,2], bins=20, density=True)

# create gaussian fit lines
gauss_fit_s = stats.norm.pdf(bins_s, gauss_s[0], gauss_s[1])
gauss_fit_d = stats.norm.pdf(bins_d, gauss_d[0], gauss_d[1])
gauss_fit_c = stats.norm.pdf(bins_c, gauss_c[0], gauss_c[1])

# plot gaussian fits
axs[0].plot(bins_s, gauss_fit_s)
axs[1].plot(bins_d, gauss_fit_d)
axs[2].plot(bins_c, gauss_fit_c)

# label mean and sd
axs[0].text(0.02, 0.97, 'mean = ' + str(round(gauss_s[0],2)), fontsize=14, transform = axs[0].transAxes)
axs[0].text(0.02, 0.94, 'sd = ' + str(round(gauss_s[1],2)), fontsize=14, transform = axs[0].transAxes)
axs[1].text(0.02, 0.97, 'mean = ' + str(round(gauss_d[0],2)), fontsize=14, transform = axs[1].transAxes)
axs[1].text(0.02, 0.94, 'sd = ' + str(round(gauss_d[1],2)), fontsize=14, transform = axs[1].transAxes)
axs[2].text(0.02, 0.97, 'mean = ' + str(round(gauss_c[0],2)), fontsize=14, transform = axs[2].transAxes)
axs[2].text(0.02, 0.94, 'sd = ' + str(round(gauss_c[1],2)), fontsize=14, transform = axs[2].transAxes)

# plot best model values
axs[0].plot([best_model[0], best_model[0]],[0, axs[0].get_ylim()[1]], color="red")
axs[1].plot([best_model[1], best_model[1]],[0, axs[1].get_ylim()[1]], color="red")
axs[2].plot([best_model[2], best_model[2]],[0, axs[2].get_ylim()[1]], color="red")

axs[0].set_title('Slip rate', fontsize=16)
axs[1].set_title('Locking depth', fontsize=16)
axs[2].set_title('Offset', fontsize=16)

axs[0].set_xlabel('Slip rate (mm/yr)', fontsize=14)
axs[1].set_xlabel('Locking depth (km)', fontsize=14)
axs[2].set_xlabel('Offset (mm/yr)', fontsize=14)

fig.tight_layout()
plt.show()

As we saw in the "walker" plot, there is a trade off between slip rate and locking depth, with an increase in one producing an increase in the other.

Lets replot the histograms above while also show the distribution of each model parameter against the other. This should make correlations between model parameters clearer to see. We'll use the Seaborn package for this, as it has a pre-made function for generate these multivariate plots. Its well worth an investigate if you want to use python to plot more scientific data in future.

正如我们在“walker”图中看到的那样，滑移率和闭锁深度之间存在权衡，其中一个的增加会导致另一个的增加。

让我们重新绘制上面的直方图，同时也显示每个模型参数相对于其他模型参数的分布。这将使模型参数之间的相关性更清晰可见。我们将使用Seaborn包，因为它有一个用于生成这些多变量图的预制函数。如果你想在未来使用python绘制更多的科学数据，这是非常值得研究的。

In [ ]:
# This plot can take a couple minutes to run (dependent on the number of iterations)

# multivariate plot
models_df = pd.DataFrame(data=models_saved, columns=["Slip rate (mm/yr)", "Locking depth (km)", "Offset (mm/yr)"])
g = sns.PairGrid(models_df, corner=True)
g.map_diag(sns.histplot)
g.map_lower(sns.kdeplot) # use this to plot as contours instead (runs slower)
g.fig.set_size_inches(13,13)

# add red dots for our best model
g.axes[1,0].scatter(best_model[0], best_model[1], c='red')
g.axes[2,0].scatter(best_model[0], best_model[2], c='red')
g.axes[2,1].scatter(best_model[1], best_model[2], c='red')

plt.show()

In [ ]:
# compare just slip rate and locking depth
models_df = pd.DataFrame(data=models_saved[:,0:2], columns=["Slip rate (mm/yr)", "Locking depth (km)"])
ax = sns.jointplot(data=models_df, x="Slip rate (mm/yr)", y="Locking depth (km)", kind="hex", height=8)
plt.show()

## 7. Extension - Fault creep 延伸点 断层蠕滑

So far we've assumed that the NAF is locked from the surface down to a given depth, below which the fault is slipping aseismically. This gives a fault-parallel velocity of zero at the fault trace. Depending on where you have drawn your profile lines so far, you may have noticed a step in the velocities across the fault that is larger than the noise level. This may be fault creep, where the fault is also slipping aseismically at a shallow depth up to the surface. This fault creep occurs alongside the deeper interseismic slip.

到目前为止，我们已经假设NAF从地表向下锁定到一个给定的深度，在这个深度以下，断层正在进行抗震滑动。这使得断层轨迹处的断层平行速度为零。到目前为止，根据您绘制剖面线的位置，您可能已经注意到断层上的速度阶跃大于噪声水平。这可能是断层蠕变，断层也在浅层向地表进行抗震滑动。这种断层蠕变与较深的震间滑动同时发生。

A schematic diagram of fault creep is shown in Figure 5.断层蠕变的示意图如图5所示。

<img src="figures/faultCreep.png" style="float: centre;" width="500"/>

*Figure 5: (a) Schematic diagram of our fault creep model. (b) Forward model showing how the fault creep is expected to change our observed signal. From Hussian et al. (2016).*

*图5:(a)我们的断层蠕变模型示意图。(b)显示断层蠕变如何改变我们观测信号的正演模型。*

We can model a combination of interseismic strain accumulation and fault creep to calculate a fault parallel velocity, $v$, at fault-perpendicular distance, $x$, using Equation 4:

我们可以对地震间应变积累和断层蠕变的组合进行建模，以计算断层垂直距离x处的断层平行速度v，使用公式4:

\begin{equation}
v(x) = -\frac{s}{\pi}arctan\left(\frac{x+x_c}{d_1}\right) + C\left[\frac{1}{\pi}arctan\left(\frac{x+x_c}{d_2}\right) - H(x+x_c)\right] + c \qquad \qquad (4)
\end{equation}

We retain the screw dislocation model from before, where slip, $s$ occurs below a depth, $d_1$. For the shallow creep, occuring at a rate of $C$ between the surface and depth $d_2$, we use a back slip approach. This models the creep as the sum of slip on the entire fault plane using a Heaviside function ($H$) plus a screw dislocation in the opposite sense to the plate motion at depth $d_2$. We also retain the static offset, $c$, and include a new offset to the fault location, $x_c$. Sometimes the observed signal may not exactly align with the mapped fault trace, and so it can be useful to include a small offset to the location of the fault.

我们保留了之前的螺位错模型，其中滑移$s$发生在深度$d_1$以下。对于表面和深度之间以C$速率发生的浅蠕变，我们采用后滑移方法。该模型使用Heaviside函数($H$)加上与深度$d_2$的板块运动相反意义的螺旋位错，将蠕变建模为整个断面上的滑动总和。我们还保留了静态偏移量$c$，并在故障位置包含了一个新的偏移量$x_c$。有时观察到的信号可能与映射的故障轨迹不完全一致，因此在故障位置上包含一个小的偏移可能是有用的。

Let's take a new profile across a creeping part of the NAF.让我们对NAF的一部分进行新的分析。

In [ ]:
# creep
prof_start = (508, 4450)
prof_end = (480, 4620)

prof_params = {
    "nbins": 100, # number of bins to split the profile into
    "width": 5  # total width of the profile in km
}

# run the profiler
bin_val, prof_bin_mids, points_val, points_dist, points_poly \
    = lib.profile_data(xx_utm,yy_utm,vel_para,prof_start,prof_end,prof_params)

# calculate profile-fault intersection angle
intersect_dist, intersect_angle = lib.profile_fault_intersection(prof_start,prof_end,fault_trace_utm)

# plot
fig, axs = plt.subplots(1,2,figsize=(15,5))

# East velocities
im = axs[0].scatter(xx_utm.flatten(), yy_utm.flatten(), s=2, c=vel_para.flatten(), cmap=cm.vik, vmin=-20, vmax=20)
axs[0].plot(fault_trace_utm[:,0], fault_trace_utm[:,1], color="red")
axs[0].plot([prof_start[0], prof_end[0]], [prof_start[1], prof_end[1]], color="red")
axs[0].plot(points_poly[:,0],points_poly[:,1], color="red")
axs[0].scatter(prof_start[0],prof_start[1], color='red')
axs[0].scatter(prof_end[0],prof_end[1], color='black')
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad="2%")
plt.colorbar(im, cax=cax, label='Fault-parallel velocity (mm/yr)')
axs[0].set_aspect('equal', 'box')
axs[0].set_title('Fault-parallel')
axs[0].set_xlim(xlim)
axs[0].set_ylim(ylim)

# profile
axs[1].scatter(points_dist,points_val)
axs[1].plot(prof_bin_mids,bin_val,color="red")
axs[1].plot([intersect_dist, intersect_dist],[axs[1].get_ylim()[0], axs[1].get_ylim()[1]], color='grey', linestyle='dashed')
axs[1].text(0.02, 0.90, 'intersection angle = ' + str(round(np.abs(intersect_angle))) + ' degrees', fontsize=14, transform = axs[1].transAxes)
axs[1].set_xlabel("Distance")
axs[1].set_ylabel("Fault-parallel velocity (mm/yr)")
axs[1].yaxis.set_label_position("right")
axs[1].yaxis.tick_right()

plt.tight_layout(w_pad=4)
plt.show()

Next, we'll run a forward model.然后运行之前的模型


In [ ]:
# Slip and locking depth
#####################################
s1 = 20 # mm/yr
s2 = 7 # mm/yr
d1 = 15 # km
d2 = 5 # km
c = -7 # mm/yr
xc = 0 # km
#####################################

# shift profile so thast fault is at 0 km
x_prof = points_dist - intersect_dist

# Vector of positions every 1 km for -100 km to 100 km
x = np.arange(-80,81,0.1)

# run the forward model
v = lib.fault_creep(x*1000, s1/1000, s2/1000, d1*1000, d2*1000, c/1000, xc=xc*1000)

# calculate rms misfit by running the forward model for x positions with velocities
v_forward = lib.fault_creep(x_prof*1000, s1/1000, s2/1000, d1*1000, d2*1000, c/1000, xc=xc*1000)
rms_forward = lib.rms_misfit(points_val,v_forward*1000)

# Plot comparison
fig, axs = plt.subplots(1,1,figsize=(15,8))

plt.scatter(x_prof,points_val)
plt.plot(x, v*1000, c='r')
plt.plot([0, 0],[axs.get_ylim()[0], axs.get_ylim()[1]], color='grey', linestyle='dashed')

plt.text(0.02, 0.9, 'slip = ' + str(s1) + ' mm/yr', fontsize=14, transform = axs.transAxes)
plt.text(0.25, 0.9, 'creep = ' + str(s2) + ' mm/yr', fontsize=14, transform = axs.transAxes)
plt.text(0.02, 0.84, 'locking depth = ' + str(d1) + ' km', fontsize=14, transform = axs.transAxes)
plt.text(0.25, 0.84, 'creep depth = ' + str(d2) + ' km', fontsize=14, transform = axs.transAxes)
plt.text(0.02, 0.78, 'offset = ' + str(c) + ' mm/yr', fontsize=14, transform = axs.transAxes)
plt.text(0.02, 0.65, 'RMS misfit = ' + str(round(rms_forward,3)) + ' mm/yr', fontsize=14, transform = axs.transAxes)

plt.xlabel('Perp. distance from fault (km)')
plt.ylabel('Fault-parallel velocity (mm/yr)')

plt.show()

## 8. Extension - Strain Rate 延伸点 应变率

Estimates of strain rate can highligh areas of concentrated deformation, an important product for understanding both seismic hazard the wider-scale kinematics. The state of strain in the lithosphere can be described using a strain rate tensor, $\dot{\varepsilon}$:

应变率的估计可以突出集中变形的区域，这是了解地震危害和更广泛的运动学的重要产品。岩石圈的应变状态可以用应变速率张量$\dot{\varepsilon}$来描述:

\begin{equation}
    \dot{\varepsilon} = \begin{bmatrix}
    \frac{\partial V_E}{\partial x} & \frac{1}{2}\left(\frac{\partial V_E}{\partial y}+\frac{\partial V_N}{\partial x}\right) & \frac{1}{2}\left(\frac{\partial V_E}{\partial z}+\frac{\partial V_U}{\partial x}\right) \\    
    \frac{1}{2}\left(\frac{\partial V_N}{\partial x}+\frac{\partial V_E}{\partial y}\right) & \frac{\partial V_N}{\partial y} & \frac{1}{2}\left(\frac{\partial V_N}{\partial z}+\frac{\partial V_U}{\partial y}\right) \\    
    \frac{1}{2}\left(\frac{\partial V_U}{\partial x}+\frac{\partial V_E}{\partial z}\right) & \frac{1}{2}\left(\frac{\partial V_U}{\partial y}+\frac{\partial V_N}{\partial z}\right) & \frac{\partial V_U}{\partial z}    
    \end{bmatrix} \qquad \qquad (5)
\end{equation}

where $v_E$, $v_N$, and $v_U$ are the velocities in the Cartesian coordinate direction, $x$, $y$, and $z$.

In this instance, we have fault-parellel velocities at distances perpendicular to the fault, and so we can only estimate the shear strain across the fault. For our screw dislocation model, we can forward model the shear strain, $e_{shear}$, using (Savage & Burford, 1973):

其中$v_E$， $v_N$和$v_U$是笛卡尔坐标方向上的速度，$x$， $y$和$z$。

在这种情况下，我们在垂直于断层的距离上有断层平行速度，因此我们只能估计断层上的剪切应变。对于我们的螺位错模型，我们可以使用(Savage & Burford, 1973)对剪切应变$e_{shear}$进行正演模拟:

\begin{equation}
e_{shear} = \left(\frac{sd}{2\pi}\right)\left(x^2+d^2\right)^{-1} \qquad \qquad (6)
\end{equation}

Let's start by calculating the shear rate for the screw dislocation model that we fit to our original profile, using both a forward model (Equation 6) and by calculating the gradient in the modelled velocities.首先，我们使用正演模型(公式6)和计算模拟速度的梯度，计算符合原始剖面的螺旋位错模型的剪切速率。

In [ ]:
# Slip rate and locking depth
#####################################
s = 19 # mm/yr
d = 11 # km
c = -5.7 # mm/yr
#####################################

# Original profile line
prof_start = (573, 4450)
prof_end = (540, 4595)
prof_params = {
    "nbins": 100, # number of bins to split the profile into
    "width": 5  # total width of the profile in km
}

# run the profiler
bin_val, prof_bin_mids, points_val, points_dist, points_poly \
    = lib.profile_data(xx_utm,yy_utm,vel_para,prof_start,prof_end,prof_params)

# calculate profile-fault intersection angle
intersect_dist, intersect_angle = lib.profile_fault_intersection(prof_start,prof_end,fault_trace_utm)

# shift profile so that the fault is at 0 km
x_prof = points_dist - intersect_dist

# Vector of positions every 1 km for -100 km to 100 km
x = np.arange(-100,101,1)

# run the forward model
v = lib.screw_disc(x*1000, s/1000, d*1000, c/1000)

# calculate shear strain rate using Equation 6
e_shear = lib.shear_strain(x*1000, s/1000, d*1000)

# also calculate shear strain rate based purely on velocity gradient
e_shear_grad = np.gradient(v,x*1000)

# Plot the original profile and model, and strain rate
fig, axs = plt.subplots(2,1,figsize=(15,15))

axs[0].scatter(x_prof, points_val)
axs[0].plot(x, v*1000, c='r')
axs[0].plot([0, 0],[axs[0].get_ylim()[0], axs[0].get_ylim()[1]], color='grey', linestyle='dashed')
axs[0].text(0.02, 0.90, 'intersection angle = ' + str(round(np.abs(intersect_angle))) + ' degrees', fontsize=14, transform = axs[0].transAxes)
axs[0].set_xlabel("Distance from fault (km)")
axs[0].set_ylabel("Fault-parallel velocity (mm/yr)")
axs[0].yaxis.set_label_position("right")
axs[0].yaxis.tick_right()

axs[1].plot(x, e_shear, c='r', label='Forward model')
axs[1].plot(x, e_shear_grad, c='b', label='Velocity gradient')
axs[1].plot([0, 0],[axs[1].get_ylim()[0], axs[1].get_ylim()[1]], color='grey', linestyle='dashed')
axs[1].legend(fontsize='x-large')
axs[1].set_xlabel("Distance from fault (km)")
axs[1].set_ylabel("Shear strain rate (/yr)")
axs[1].yaxis.set_label_position("right")
axs[1].yaxis.tick_right()

We can get nice smooth estimates of strain rate from our screw dislocation models. In real studies, we often want to estimate strain rates directly from our velocities, as this will capture signals that are not represented in our model. However, we still require some degree of smoothing to suppress short-wavelength noise in the InSAR. This noise, while small compared to the tectonic signal, can cause large gradients between adjacent points that then drown out the underying tectonic strain rate.

Let's plot both the strain rate directly calculated from the binned profiled fault-parallel velocities, and one calculated from the same velocities but smoothed with a sliding window average. Try changing the size of the smoothing window.

我们可以从螺位错模型中得到应变率的平滑估计。在实际的研究中，我们经常希望直接从我们的速度中估计应变率，因为这将捕获在我们的模型中没有表示的信号。然而，我们仍然需要一定程度的平滑来抑制InSAR中的短波噪声。这种噪音虽然与构造信号相比很小，但会在相邻点之间造成很大的梯度，从而淹没了下面的构造应变率。

让我们绘制直接由分层断层平行速度计算的应变率，以及由相同速度计算但用滑动窗口平均平滑的应变率。尝试更改平滑窗口的大小。

In [ ]:
# Window size for smoothing, must be an odd number
#####################################
wind_size = 9
#####################################

# shift fault location to 0 km
x_prof = prof_bin_mids - intersect_dist

# calcualte gradient of original binned velocities
e_shear = np.gradient(bin_val/1000, x_prof*1000)

# apply a sliding mean window to smooth out some of the noise
x_smoothed, e_shear_smoothed = lib.sliding_window_mean(x_prof, bin_val, 11)

# calculated the gradient from the smoothed velocities
e_shear_smoothed = np.gradient(e_shear_smoothed/1000, x_smoothed*1000)

# Plot comparison
fig, axs = plt.subplots(1,1,figsize=(15,8))

plt.plot(x_prof, e_shear, c='r', label='Original binned velocities')
plt.plot(x_smoothed, e_shear_smoothed, c='b', label='Smoothed velocities')
plt.plot([0, 0],[axs.get_ylim()[0], axs.get_ylim()[1]], color='grey', linestyle='dashed')
plt.xlabel('Distance from fault (km)')
plt.ylabel('Shear strain rate (/yr)')
plt.legend(fontsize="x-large")

plt.show()

If we calculate the strain rate directly from the binned velocities (which have themselves already been smoothed to some degree), then the strain signal we would expect to see based on Equation 5 is completely masked.

By applying a smoothing window (a window size of 9 works quite well), we can bring out the higher strain across the fault trace. If the window size is increased too far, then we can actually smooth out the original interseismic signal, and this peak disappears again. How we filter out high-frequency noise in the velocities is integral to our estimates of strain rate.

For further reading, I recommend Weiss et al. (2020), who apply the Velmap method to all of Turkey, and Ou et al. (2022), who use a median filter method to localise strain onto a number of faults in the Tibetan Plateau.

如果我们直接从分组速度(它们本身已经在某种程度上平滑了)中计算应变速率，那么我们期望看到的基于公式5的应变信号是完全被掩盖的。

通过应用平滑窗口(窗口大小为9相当好)，我们可以在故障轨迹上得到更高的应变。如果窗口大小增加得太大，那么我们实际上可以平滑原始的震间信号，这个峰值再次消失。我们如何过滤掉速度中的高频噪声是我们对应变率估计的组成部分。

为了进一步阅读，我推荐Weiss等人(2020)，他们将Velmap方法应用于整个土耳其，以及Ou等人(2022)，他们使用中值滤波方法将应变定位到青藏高原的一些断层上。

## Conclusion

Hopefully, you now have a basic understanding of how we measure and model interseismic deformation. This practical has been shared under an open-access licence (see repo), so feel free to pass it on and utilise any of the underlying code. 

希望你们现在对我们如何测量和模拟震间变形有了基本的了解。本实用程序已在开放访问许可下共享(参见repo)，因此请随意传递并使用任何底层代码。

If you are interested in performing velocity decompositions on a larger scale, have a look at our open-access Matlab package:

如果您有兴趣在更大范围内执行速度分解，请查看我们的开放访问Matlab软件包:
https://github.com/andwatson/decompose_insar_velocities

If you are interested in a more developed version of the bayesian inversion, have a look at both Hussain et al. (2016) and the GBIS inversion software, for interseismic and coseismic deformation, respectively:

如果你对贝叶斯反演的更先进版本感兴趣，可以看看Hussain等人(2016)和GBIS反演软件，分别用于地震间和同震变形:
https://comet.nerc.ac.uk/gbis/

如果你在这个实践中发现任何问题，请将它们记录在github存储库的“问题”下。

Andrew Watson - 2022

### References

Elliott, J. (2009). Strain accumulation & release on the Tibetan Plateau measured using InSAR (Doctoral dissertation, Oxford University).

Hussain, E., Wright, T. J., Walters, R. J., Bekaert, D., Hooper, A., & Houseman, G. A. (2016). Geodetic observations of postseismic creep in the decade after the 1999 Izmit earthquake, Turkey: Implications for a shallow slip deficit. Journal of Geophysical Research: Solid Earth, 121(4), 2980-3001.

Ou, Q., Daout, S., Weiss, J. R., Shen, L., Lazecký, M., Wright, T. J., & Parsons, B. E. (2022). Large‐Scale Interseismic Strain Mapping of the NE Tibetan Plateau From Sentinel‐1 Interferometry. Journal of Geophysical Research: Solid Earth, 127(6), e2022JB024176.

Savage, J. C., & Burford, R. O. (1973). Geodetic determination of relative plate motion in central California. Journal of Geophysical Research, 78(5), 832-845.

Vernant, P. (2015). What can we learn from 20 years of interseismic GPS measurements across strike-slip faults?. Tectonophysics, 644, 22-39.

Weiss, J. R., et al. (2020). High‐resolution surface velocities and strain for Anatolia from Sentinel‐1 InSAR and GNSS data. Geophysical Research Letters, 47(17), e2020GL087376.